In [1]:
#r ".\binaries\BoSSSpad.dll"
#r ".\binaries\SAIDT.dll"
#r ".\binaries\ApplicationWithIDT.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Platform.LinAlg;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.Grid.RefElements;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using ApplicationWithIDT;
using SAIDT;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();


# Run Simulation

## SinglePhaseField

In [2]:
string dbPath=null;

In [5]:
using SAIDT;
BoSSS.Solution.Application.InitMPI();
BoSSS.Solution.Application.DeleteOldPlotFiles();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 50,
                    dgDegree: 4,
                    numOfCellsX: 3,
                    numOfCellsY: 3,
                    OptiNumOfCellsX: 3,
                    OptiNumOfCellsY: 3,
                    agg: 0.4,
                    ImmediatePlotPeriod: -1,
                    optiLevelSetType: OptiLevelSetType.SinglePhaseField,
                    LSDegree: 3
                    );
CSAIDT.SessionName="SAIDT_DGLevelSet";
CSAIDT.ProjectName="SAIDT_DGLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.InitializeEverything();


In [40]:
Func<double[],double> discCircA= delegate(double[] X){
    var y=X[1];
    var x=X[0];
    double result=0.0;
    if (x>1.0/3.0 &&x<2.0/3.0 &&y>1.0/3.0 &&y<2.0/3.0 )
    {
        // Apply the first function for y > 0.5
        result = x>0.5 ? 1+1*Math.Sin(100*x)*Math.Sin(100*y): 1+1*Math.Cos(100*x)*Math.Cos(100*y);
    }
    else 
    {
        // Apply the second function for y < 0.75
        result = x+y;
    }
    
    return result;
};
Func<double[],double> discCircB= delegate(double[] X){
    var y=X[1];
    var x=X[0];
    double result=0.0;
    if (x>1.0/3.0 &&x<2.0/3.0 &&y>1.0/3.0 &&y<2.0/3.0 )
    {
        // Apply the first function for y > 0.5
        result = x< 0.5 ? 4+ 1*Math.Sin(100*x)*Math.Sin(100*y): 4+1*Math.Cos(100*x)*Math.Cos(100*y);
    }
    else 
    {
        // Apply the second function for y < 0.75
        result = 4;
    }
    
    return result;
};

In [47]:
using BoSSS.Solution.Tecplot;
using System.IO;
Directory.GetFiles(".", "*_SP_*").ForEach(file => File.Delete(file));

public void plotSF(string name){
    List<DGField> flds= new List<DGField>();
foreach(var field in pSAIDT.RegisteredFields){
    if(field is XDGField xfield){
        flds.Add(xfield.GetSpeciesShadowField("L"));
        flds.Add(xfield.GetSpeciesShadowField("R"));
        flds.Add(field);
    }else{
        flds.Add(field);
    }
}
var texplot = new Tecplot(flds[0].GridDat, 2);
texplot.PlotFields( name,0.0,flds);
}

In [48]:
BoSSS.Solution.Application.DeleteOldPlotFiles();
pSAIDT.ConservativeFields[0].GetSpeciesShadowField("L").ProjectField(discCircA);
pSAIDT.ConservativeFields[0].GetSpeciesShadowField("R").ProjectField(discCircB);
pSAIDT.Control.ApplyReiInit=true;
pSAIDT.GetPerssonSensor(false);
pSAIDT.Control.reInitTols=new double[] {-2,-2,-2,-2,-2,-2};
plotSF("reinit_0");
pSAIDT.Reinitialization();
pSAIDT.GetPerssonSensor(false);
plotSF("reinit_1");

## Spline level set

In [8]:
BoSSS.Solution.Application.InitMPI();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 100,
                    dgDegree: 0,
                    numOfCellsX: 5,
                    numOfCellsY: 5,
                    OptiNumOfCellsX: 5,
                    OptiNumOfCellsY: 5,
                    agg: 0.4,
                    ImmediatePlotPeriod: -1,
                    optiLevelSetType: OptiLevelSetType.SplineLevelSet,
                    LSDegree: 3,
                    isFarConfig:true
                    );
CSAIDT.SessionName="SAIDT_SplineLevelSet";
CSAIDT.ProjectName="SAIDT_SplineLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.RunSolverMode();
DOFS.Add("SplineLevelSet",pSAIDT.LevelSetOpti.GetLength());


## SpecFemField

In [11]:
using SAIDT;
BoSSS.Solution.Application.InitMPI();
var pSAIDT = new SAIDTMain();
var CSAIDT = SAIDTHardCodedControl.StraightShock(
                    dbPath: dbPath,
                    MaxIterations: 50,
                    dgDegree: 0,
                    numOfCellsX: 5,
                    numOfCellsY: 5,
                    OptiNumOfCellsX: 5,
                    OptiNumOfCellsY: 5,
                    agg: 0.4,
                    ImmediatePlotPeriod: 5,
                    optiLevelSetType: OptiLevelSetType.SpecFemField,
                    LSDegree: 3,
                    isFarConfig:true
                    );
CSAIDT.SessionName="SAIDT_CGLevelSet";
CSAIDT.ProjectName="SAIDT_CGLevelSet";
pSAIDT.Init(CSAIDT);
pSAIDT.RunSolverMode();
DOFS.Add("SpecFemField",pSAIDT.LevelSetOpti.GetLength());

In [12]:
pSAIDT.ResNorms.SaveToTextFile(pSAIDT.Control.ProjectName+"_ResPlot.txt");
pSAIDT.obj_f_vals.SaveToTextFile(pSAIDT.Control.ProjectName+"_EnResPlot.txt");
pSAIDT.GetResPlot().ToGnuplot().PlotSVG(xRes:600,yRes:500)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 -16 
 
 
 
 
 10 -14 
 
 
 
 
 10 -12 
 
 
 
 
 10 -10 
 
 
 
 
 10 -8 
 
 
 
 
 10 -6 
 
 
 
 
 10 -4 
 
 
 
 
 10 -2 
 
 
 
 
 10 0 
 
 
 
 
 10 2 
 
 
 
 
 0 
 
 
 
 
 5 
 
 
 
 
 10 
 
 
 
 
 15 
 
 
 
 
 20 
 
 
 
 
 25 
 
 
 
 
 30 
 
 
 
 
 35 
 
 
 
 
 40 
 
 
 
 
 45 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 Iteration 
 
 
 
 
 ||R(z)|| 
 
 
 ||R(z)|| 
 
 
 
	<path stroke='rgb( 0, 0, 255)' d='M388.5,57.1 L441.9,57.1 M53.9,77.8 L64.9,78.6 L76.0,80.1 L87.0,78.9 L98.0,80.1 L109.0,78.9
 L120.1,79.1 L131.1,80.7 L142.1,80.8 L153.1,80.8 L164.2,80.8 L175.2,81.6 L186.2,81.7 L197.2,81.8
 L208.3,82.2 L219.3,82.3 L230.3,82.4 L241.4,82.7 L252.4,82.7 L263.4,82.7 L274.4,82.7 L285.5,82.7
 L296.5,82.7 L307.5,82.7 L318.5,83.8 L329.6,81.2 L340.6,82.8 L351.6,84.8 L362.6,83.4 L373.7,86.3
 L384.7,87.4 L395.7,88.0 L406.8,88.3 L417.8,88.4 L428.8,88.5 L439.8,88.6 L450.9,88.7 L461.9,88.7
 L472.9,88.8 L483.9,88.8 L495.0,88.9 L506.0,88.9 L517.0,89.0 L528.0,89.0 '/> 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 ||r(z)|| 
 
 
 ||r(z)|| 
 
 
 
	<path stroke='rgb(255, 0, 0)' d='M388.5,81.1 L441.9,81.1 M53.9,422.7 L64.9,106.7 L76.0,108.8 L87.0,102.2 L98.0,100.6 L109.0,98.7
 L120.1,98.3 L131.1,110.8 L142.1,111.2 L153.1,111.2 L164.2,111.2 L175.2,108.1 L186.2,112.7 L197.2,112.6
 L208.3,107.6 L219.3,107.9 L230.3,107.8 L241.4,107.3 L252.4,107.3 L263.4,107.3 L274.4,107.3 L285.5,107.3
 L296.5,107.3 L307.5,107.3 L318.5,103.0 L329.6,99.7 L340.6,97.6 L351.6,97.9 L362.6,97.0 L373.7,96.4
 L384.7,104.9 L395.7,103.4 L406.8,105.1 L417.8,105.1 L428.8,105.7 L439.8,106.3 L450.9,106.2 L461.9,106.4
 L472.9,106.3 L483.9,106.4 L495.0,106.5 L506.0,106.7 L517.0,106.7 L528.0,106.8 '/>

In [13]:
var si= database.Sessions.Pick(0);
pSAIDT.PlotShadowFields(si);

In [14]:
using System.IO;
static void AppendRowToCSV(string filePath, double[] rowData)
    {
        // Create a StreamWriter in append mode to append rows to the CSV file
        using (StreamWriter writer = new StreamWriter(filePath, true))
        {
            int cols = rowData.Length;

            // Write double values of the new row to the file
            for (int i = 0; i < cols; i++)
            {
                writer.Write(rowData[i]);

                // Add a comma after each value except for the last one in a row
                if (i < cols - 1)
                {
                    writer.Write(";");
                }
            }

            // Move to the next line after writing the row
            writer.WriteLine();
        }
    }
    static void WriteCSV(string filePath, string[] data)
    {
        // Create a StreamWriter to write to the CSV file
        using (StreamWriter writer = new StreamWriter(filePath))
        {
            int cols = data.Length;

            // Loop through the data array and write to the file
                for (int j = 0; j < cols; j++)
                {
                    // Write each cell value to the file
                    writer.Write(data[ j]);

                    // Add a comma after each cell value except for the last one in a row
                    if (j < cols - 1)
                    {
                        writer.Write(";");
                    }
                }

                // Move to the next line after each row
                writer.WriteLine();
            
        }
    }
    static void DeleteCSVFiles()
    {
        try
        {
            // Get all CSV files in the specified directory
            string[] csvFiles = Directory.GetFiles("*.csv");

            // Delete each CSV file
            foreach (string file in csvFiles)
            {
                File.Delete(file);
                Console.WriteLine($"Deleted: {file}");
            }
        }
        catch (Exception ex)
        {
            Console.WriteLine($"Error deleting CSV files: {ex.Message}");
        }
    }

## Measure wall-clock time for computation of sensitivities and residuals 

In [15]:
using System.Diagnostics;
BoSSS.Solution.Application.InitMPI();
DeleteCSVFiles();

int nRefs=4;
var optLsTypes=new OptiLevelSetType[] {OptiLevelSetType.SinglePhaseField,OptiLevelSetType.SplineLevelSet,OptiLevelSetType.SpecFemField};
var CellsVsRuntime_dRdPhi = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
var CellsVsRuntime_Residuals = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
var CellsVsRuntime_dRdU = MultidimensionalArray.Create(nRefs,optLsTypes.Length +1);
string[] stringArray = new string[] { "nCells" }.Concat(optLsTypes.Select(x => x.ToString())).ToArray();

//Create files for results
string filePathdRdPhi = "TimeDRdPhi.csv";
string filePathdRdU = "TimedRdU.csv";
string filePathResiduals = "TimeResiduals.csv";
string filePathDOFs = "DOFs.csv";
WriteCSV(filePathdRdPhi,stringArray);
WriteCSV(filePathdRdU,stringArray);
WriteCSV(filePathResiduals,stringArray);
WriteCSV(filePathDOFs,stringArray);

for(int n =0; n< nRefs;n++){
    //counter for level set type
    int iLS=1;
    //save cell resolution
    int reso= (int) Math.Pow(2,n+1);
    CellsVsRuntime_dRdPhi[n,0]=reso*reso;
    double[] dRdPhiResults= new double[optLsTypes.Length +1];
    dRdPhiResults[0]=reso*reso;
    double[] dRdUResults=dRdPhiResults.CloneAs();
    double[] resiudalResults=dRdPhiResults.CloneAs();
    double[] DOFs=dRdPhiResults.CloneAs();
    foreach(OptiLevelSetType lstype in optLsTypes){
        var pSAIDT = new SAIDTMain();
        var CSAIDT = SAIDTHardCodedControl.StraightShock(
                            dbPath: null,
                            MaxIterations: 1,
                            dgDegree: 0,
                            numOfCellsX: reso,
                            numOfCellsY: reso,
                            OptiNumOfCellsX: reso,
                            OptiNumOfCellsY: reso,
                            agg: 0.4,
                            ImmediatePlotPeriod: -1,
                            optiLevelSetType: lstype,
                            LSDegree: 3,
                            isFarConfig:true);

        pSAIDT.Init(CSAIDT);    
        pSAIDT.InitializeEverything();
        //pSAIDT.RunSolverMode();

        //Measure time for creation of Jacobians dRdPhi
        Stopwatch stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.FD_LevelSet();
        stopwatch.Stop();
        CellsVsRuntime_dRdPhi[n,iLS] =stopwatch.ElapsedMilliseconds;
        dRdPhiResults[iLS]=stopwatch.ElapsedMilliseconds;

        //Measure time for creation of Jacobians dRdU
        stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.Oproblem.GetJacobians(pSAIDT.ConservativeFields, pSAIDT.XSpatialOperator.LinearizationHint);
        stopwatch.Stop(); 
        CellsVsRuntime_dRdU[n,iLS] =stopwatch.ElapsedMilliseconds;
        dRdUResults[iLS]=stopwatch.ElapsedMilliseconds;

        //Measure time for evaluation of the Residuals
        stopwatch = new Stopwatch();stopwatch.Start();
        pSAIDT.ComputeResiduals();
        stopwatch.Stop(); Console.WriteLine("Elapsed Time: " + stopwatch.ElapsedMilliseconds + "ms");
        CellsVsRuntime_Residuals[n,iLS] =stopwatch.ElapsedMilliseconds;
        resiudalResults[iLS]=stopwatch.ElapsedMilliseconds;

        DOFs[iLS]=pSAIDT.LevelSetOpti.GetLength();

        iLS++;
    }
    // Append the new row to the existing CSV file
    AppendRowToCSV(filePathdRdPhi, dRdPhiResults);
    AppendRowToCSV(filePathdRdU, dRdUResults);
    AppendRowToCSV(filePathResiduals, resiudalResults);
    AppendRowToCSV(filePathDOFs, DOFs);
}

In [16]:
public PlotFormat GetFormat(int count){
    
    var allPT = new BoSSS.Solution.Gnuplot.PointTypes[] { PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle,PointTypes.Diamond, PointTypes.LowerTriangle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle, PointTypes.Circle, PointTypes.OpenDiamond, PointTypes.OpenLowerTriangle, PointTypes.OpenCircle};
    var allC = new BoSSS.Solution.Gnuplot.LineColors[] { LineColors.Blue, LineColors.Black, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red, LineColors.Red, LineColors.Red,LineColors.Blue, LineColors.Black, LineColors.Red};
    var Fmt = new PlotFormat();
    Fmt.PointSize = 0.8;
    Fmt.LineWidth = 1;    
    Fmt.Style     = Styles.LinesPoints;

    Fmt.LineColor = allC[count];
    Fmt.PointType =  allPT[count];
    return Fmt;
}

In [17]:
CellsVsRuntime_dRdPhi

Dimension,2
Lengths,"[ 4, 4 ]"
Storage,"[ 4, 1087, 174, 911, 16, 5625, 388, 3309, 64, 29468, 1001, 19166, 256, 242434, 3516, 166633 ]"
IsContinuous,True
StructureType,General
Length,16
NoOfCols,4
NoOfRows,4
IsLocked,False


In [18]:
var plot = new Plot2Ddata();
int count=0;
for(int iLS=0;iLS<CellsVsRuntime_dRdPhi.Lengths[1]-1; iLS ++){
    plot.AddDataGroup(optLsTypes[iLS].ToString(),CellsVsRuntime_dRdPhi.ExtractSubArrayShallow(-1,0).To1DArray(),CellsVsRuntime_dRdPhi.ExtractSubArrayShallow(-1,iLS+1).To1DArray(), GetFormat(count));
    count++;
}
plot.Xlabel="number of Cells";
plot.LogY = true;
plot.LogX = true;
plot.ShowXtics = true;
plot.ToGnuplot().PlotSVG(xRes:1200,yRes:600)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 10 4 
 
 
 
 
 10 5 
 
 
 
 
 10 6 
 
 
 
 
 10 0 
 
 
 
 
 10 1 
 
 
 
 
 10 2 
 
 
 
 
 10 3 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 number of Cells 
 
 
 
 
 SinglePhaseField 
 
 
 SinglePhaseField 
 
 
 
 
 
 
 
 
 
 
 SplineLevelSet 
 
 
 SplineLevelSet 
 
 
 
 
 
 
 
 
 
 
 SpecFemField 
 
 
 SpecFemField